In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from ca_utils import ResNet, BasicBlock
import os

torch.manual_seed(100)#chnge to 100
# Initialize the model
num_classes = 10  # Number of classes in your dataset
model = ResNet(block=BasicBlock, layers=[1, 1, 1], num_classes=num_classes)
model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

import os  # Make sure to import os

image_transforms = {
    'train': transforms.Compose([
        # transforms.RandomResizedCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5],
        transforms.RandomHorizontalFlip(0.5))
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ]),
}

# Load the training and validation datasets with new transformations
train_data = datasets.ImageFolder(r"C:\Users\risha\Downloads\train", transform=image_transforms['train'])
validation_data = datasets.ImageFolder(r"C:\Users\risha\Downloads\val", transform=image_transforms['val'])

# Create data loaders
train_loader = DataLoader(train_data, batch_size=64, shuffle=True, num_workers=4,pin_memory=True)
validation_loader = DataLoader(validation_data, batch_size=64, shuffle=False, num_workers=4,pin_memory=True)
# Training function
def train_cnn(model, train_loader, validation_loader, device, num_epochs=50):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    # scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

    best_val_accuracy = 0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Validation loop
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in validation_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_accuracy = correct / total
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            # Check if the directory exists, if not, create it
            if not os.path.exists("data"):
                os.makedirs("data")
            torch.save(model.state_dict(), "data/weights_resnet(q4).pth")  # Save best model weights
        if epoch%10==0:
            torch.save(model.state_dict(), f"./weights_resnet_{epoch}.pth")
        print(f'Epoch {epoch + 1}, Loss: {running_loss}, Validation Accuracy: {val_accuracy:.2f}%')
        # scheduler.step()

# Train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_cnn(model, train_loader, validation_loader, device)

Epoch 1, Loss: 224.67632400989532, Validation Accuracy: 0.32%
Epoch 2, Loss: 195.1426076889038, Validation Accuracy: 0.40%
Epoch 3, Loss: 182.87325501441956, Validation Accuracy: 0.45%
Epoch 4, Loss: 172.9042911529541, Validation Accuracy: 0.45%
Epoch 5, Loss: 163.05141961574554, Validation Accuracy: 0.51%
Epoch 6, Loss: 155.67641657590866, Validation Accuracy: 0.51%
Epoch 7, Loss: 148.64063793420792, Validation Accuracy: 0.54%
Epoch 8, Loss: 144.90317052602768, Validation Accuracy: 0.54%
Epoch 9, Loss: 139.40835803747177, Validation Accuracy: 0.57%
Epoch 10, Loss: 134.87102526426315, Validation Accuracy: 0.53%
Epoch 11, Loss: 132.5189408659935, Validation Accuracy: 0.54%
Epoch 12, Loss: 127.9211415052414, Validation Accuracy: 0.59%
Epoch 13, Loss: 123.45229178667068, Validation Accuracy: 0.58%
Epoch 14, Loss: 121.22440630197525, Validation Accuracy: 0.57%
Epoch 15, Loss: 118.49577730894089, Validation Accuracy: 0.62%
Epoch 16, Loss: 115.74443608522415, Validation Accuracy: 0.51%
Epoch